<h1>Journeys in reversing a string</h1>
<p>Early on when I was learning to code, I was watching a youtube video of a coding interview and the opening question was "how would you reverse a string?"</p>
<p>It's a very simple task, but I gather that the interviewer asked it because it is a good test case to demonstrate understanding of the basic workings of Python's central data structures.</p>
<p>I've come to like it because it's a quick exercise to try out new ideas as I become comfortable with them, as you will see below.</p>

In [110]:
import string
import random
from timeit import timeit

#This function makes it easy to test each iteration 
#with timeit while keeping the functions pure.
def close_over_string(func, string):
    return lambda: func(string)

#This creates a random 100,000 char long string to test the reverse.
def generate_word(length):    
    letters = string.ascii_lowercase
    word = []
    for i in range(length):
        word.append(random.choice(letters))
    return ''.join(word)

word = generate_word(100000)

<h2>String Concatenation</h2>
<p>The first solution offered in the video looked something like this.</p>

In [99]:
def reverse(string):
    reverse_string = ''
    for i in range(len(string)):
        next_char = string[i]
        reverse_string = next_char + reverse_string
        
    return reverse_string

harv_rev = close_over_string(reverse, word)
timeit(harv_rev, number=1000)

325.8597129200007

<p>This does work, but there is a detail that keeps it from being the best option. Strings are immutable in Python, which means when you do concatenation with the "+" operator, each character in "reverse_string" must be iterated through as it is added. This makes the complexity tend toward n<sup>2</sup>. As the test shows, the time it takes to run this on a large string is <i>long</i>.</p>
<h2>Iteration with lists</h2>
<p>When I write the string reverse function explicitly with simple iteration, this is what I come up with:</p> 

In [97]:
def reverse_while(string):
    lst = list(string)
    rev = []
    while lst:
        next_char = lst.pop()
        rev.append(next_char)
    return ''.join(rev)

rev_while = close_over_string(reverse_while, word)
timeit(rev_while, number=1000)

18.737893658999383

<p>As you can see, much faster, but more importantly it uses the stack-like character of Python lists to assemble the string backwards, so linear time.</p>
<p>Next I tried an iteration with a classic style for loop with an index (though running backwards), to see if slicing into the string was faster than first casting to a list.</p>

In [98]:
def reverse_index(string):
    rev = []
    for i in range(len(string),0, -1):
        rev.append(string[i - 1])
    return ''.join(rev)

rev_idx = close_over_string(reverse_index, word)
timeit(rev_idx, number=1000)

21.962176830998942

<p>Turns out, no, it's not.</p>

In [70]:
timeit(reverse_c, number = 1000)

2.0472326929998417

<h2>Recursion</h2>
<p>Doing this task with recursion I would argue makes for a cleaner statement of the task than iterative or string concatenation. However, recursion isn't great in Python. It will give a max recursion depth error (though this is adjustable), and it's kind of slow. When I time the recursive reverse against even the n<sup>2</sup> string concat it doesn't hold up. Not sure if it's only recursion or if there is another reason why this takes so long.</p>

In [116]:
short_word = generate_word(2000)

def recursive_rev(string):
    if string:
        return string[-1] + recursive_rev(string[:-1])
    return ''

rec_rev = close_over_string(recursive_rev, short_word)
rev_while = close_over_string(reverse_while, short_word)
rev_concat = close_over_string(reverse, short_word)

print("Recursive Reverse")
print(timeit(rec_rev, number = 1000))
print("Iterative Reverse")
print(timeit(rev_while, number = 1000))
print("String Concat Reverse")
print(timeit(rev_concat, number = 1000))

Recursive Reverse
1.8846817379999266
Iterative Reverse
0.34920748500007903
String Concat Reverse
0.7148201039999549


<p>ADD TAIL RECURSION EXAMPLE</p>

<h2>Slicing (or how I would reverse a string IRL)</h2>
<p>Python's slicing feature is I think the cleanest, and certainly the most performant way to approach this problem. Arguably It sacrifices readibility and explicitness, but most Python programmers that I interact with are very familiar with slicing.</p>

In [124]:
def reverse_slice(string):
    return string[::-1]

rev_slice = close_over_string(reverse_slice, word)

timeit(rev_slice, number=1000)

0.0547043090009538

<h2>Exploring Laziness</h2>
<p>